# **Colab Notebook Sample Template**


In [ ]:
!rm -rf nail-feature-classifier
!git clone https://github.com/Shibarashii/nail-feature-detection.git
%cd nail-feature-detection

Install `requirements.txt`


In [ ]:
import subprocess
import pkg_resources

with open("requirements.txt") as f:
    for line in f:
        pkg = line.strip()
        if not pkg or pkg.startswith("#"):
            continue
        pkg_name = pkg.split("==")[0]
        try:
            pkg_resources.get_distribution(pkg_name)
        except pkg_resources.DistributionNotFound:
            print(f"Installing missing package: {pkg}")
            subprocess.check_call(["pip", "install", pkg])

## Training


In [ ]:
!PYTHONPATH=. python scripts/train.py \
    --model yolov11s \
    --data config/data.yaml \
    --epochs 150 \
    --batch 64 \
    --imgsz 416 \
    --lr0 0.01 \
    --patience 50 \
    --mosaic 1.0 \
    --mixup 0.1 \
    --save-period 20 \
    --name nail_detector_yolov11s

## Evaluation


In [ ]:
!PYTHONPATH=. python scripts/evaluate.py --weights outputs/models/nail_detector_yolov11s/weights/best.pt --split val

## Download Outputs


In [ ]:
import shutil
from pathlib import Path
from google.colab import files
import zipfile
from datetime import datetime


def download_all_outputs(experiment_name=None):
    """
    Download all training outputs as a single zip file

    Args:
        experiment_name: Specific experiment folder name, or None for all
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    if experiment_name:
        source_dir = Path(f"outputs/models/{experiment_name}")
        zip_name = f"{experiment_name}_{timestamp}.zip"
    else:
        source_dir = Path("outputs")
        zip_name = f"all_outputs_{timestamp}.zip"

    if not source_dir.exists():
        print(f"❌ Directory not found: {source_dir}")
        return

    print(f"📦 Creating zip file: {zip_name}")
    print(f"📁 Source: {source_dir}")

    shutil.make_archive(
        zip_name.replace('.zip', ''),
        'zip',
        source_dir.parent,
        source_dir.name
    )

    print(f"⬇️  Downloading {zip_name}...")
    files.download(zip_name)
    print("✅ Download complete!")


download_all_outputs()